In [10]:
import sqlite3
import pandas as pd
from datetime import datetime


# Kết nối đến database
conn = sqlite3.connect('Temp_database/zp01.db')


# Định nghĩa khoảng thời gian
start_time = '2025-06-14 15:55:22.736839+07:00'
end_time = '2025-06-15 23:21:37.408346+07:00'


print(f"📊 Phân tích PER, RF Quality và CCA Competition cho từng kit từ {start_time} đến {end_time}")
print("🔍 Chỉ tính RSSI/LQI từ RX_ENDED packets (successful reception)")
print("📡 CCA Competition = TX_CCA_BUSY/TX_ENDED (mức độ cạnh tranh truy cập môi trường)")
print("=" * 100)


# Query để lấy dữ liệu PER cho từng kit
per_query = """
SELECT 
    ID as Kit_ID,
    event, 
    COUNT(*) as count
FROM zigbee_packets 
WHERE created_at >= ? AND created_at <= ?
AND event IN ('RX_CORRUPTED', 'RX_ENDED')
AND ID IS NOT NULL 
AND ID != ''
GROUP BY ID, event
ORDER BY ID, event
"""


# Query riêng để lấy RSSI/LQI CHỈ TỪ RX_ENDED packets
rf_quality_query = """
SELECT 
    ID as Kit_ID,
    COUNT(*) as success_count,
    MIN(rssi) as min_rssi,
    MAX(rssi) as max_rssi,
    AVG(rssi) as avg_rssi,
    MIN(lqi) as min_lqi,
    MAX(lqi) as max_lqi,
    AVG(lqi) as avg_lqi
FROM zigbee_packets 
WHERE created_at >= ? AND created_at <= ?
AND event = 'RX_ENDED'
AND ID IS NOT NULL 
AND ID != ''
GROUP BY ID
ORDER BY ID
"""


# THÊM MỚI: Query để lấy dữ liệu TX cho CCA Competition
tx_cca_query = """
SELECT 
    ID as Kit_ID,
    event, 
    COUNT(*) as count
FROM zigbee_packets 
WHERE created_at >= ? AND created_at <= ?
AND event IN ('TX_CCA_BUSY', 'TX_ENDED')
AND ID IS NOT NULL 
AND ID != ''
GROUP BY ID, event
ORDER BY ID, event
"""


# Thực hiện query cho PER
cursor = conn.cursor()
cursor.execute(per_query, (start_time, end_time))
per_results = cursor.fetchall()


# Thực hiện query cho RF Quality (chỉ RX_ENDED)
cursor.execute(rf_quality_query, (start_time, end_time))
rf_results = cursor.fetchall()


# THÊM MỚI: Thực hiện query cho TX CCA
cursor.execute(tx_cca_query, (start_time, end_time))
tx_cca_results = cursor.fetchall()


# Tạo dictionary để lưu kết quả PER
kit_per_data = {}
for kit_id, event, count in per_results:
    if kit_id not in kit_per_data:
        kit_per_data[kit_id] = {'RX_CORRUPTED': 0, 'RX_ENDED': 0}
    kit_per_data[kit_id][event] = count


# Tạo dictionary để lưu kết quả RF Quality (chỉ từ RX_ENDED)
kit_rf_data = {}
for kit_id, success_count, min_rssi, max_rssi, avg_rssi, min_lqi, max_lqi, avg_lqi in rf_results:
    kit_rf_data[kit_id] = {
        'success_count': success_count,
        'min_rssi': min_rssi if min_rssi is not None else 0,
        'max_rssi': max_rssi if max_rssi is not None else 0,
        'avg_rssi': round(avg_rssi, 2) if avg_rssi is not None else 0,
        'min_lqi': min_lqi if min_lqi is not None else 0,
        'max_lqi': max_lqi if max_lqi is not None else 0,
        'avg_lqi': round(avg_lqi, 2) if avg_lqi is not None else 0
    }


# THÊM MỚI: Tạo dictionary để lưu kết quả TX CCA
kit_tx_cca_data = {}
for kit_id, event, count in tx_cca_results:
    if kit_id not in kit_tx_cca_data:
        kit_tx_cca_data[kit_id] = {'TX_CCA_BUSY': 0, 'TX_ENDED': 0}
    kit_tx_cca_data[kit_id][event] = count


print("📈 Thống kê packets, RF Quality và CCA Competition theo từng kit:")
print("-" * 90)
for kit_id in set(list(kit_per_data.keys()) + list(kit_tx_cca_data.keys())):
    per_data = kit_per_data.get(kit_id, {'RX_CORRUPTED': 0, 'RX_ENDED': 0})
    rf_data = kit_rf_data.get(kit_id, {})
    tx_cca_data = kit_tx_cca_data.get(kit_id, {'TX_CCA_BUSY': 0, 'TX_ENDED': 0})
    
    rx_corrupted = per_data['RX_CORRUPTED']
    rx_ended = per_data['RX_ENDED']
    total_rx = rx_corrupted + rx_ended
    
    tx_cca_busy = tx_cca_data['TX_CCA_BUSY']
    tx_ended = tx_cca_data['TX_ENDED']
    total_tx = tx_cca_busy + tx_ended
    
    print(f"📱 Kit {kit_id}:")
    print(f"   📦 RX_CORRUPTED: {rx_corrupted:,}")
    print(f"   📦 RX_ENDED: {rx_ended:,}")
    print(f"   📦 Total RX: {total_rx:,}")
    print(f"   📡 TX_CCA_BUSY: {tx_cca_busy:,}")
    print(f"   📡 TX_ENDED: {tx_ended:,}")
    print(f"   📡 Total TX: {total_tx:,}")
    
    # RSSI/LQI info CHỈ TỪ RX_ENDED (successful packets)
    if rf_data and rx_ended > 0:
        print(f"   📡 RSSI (Success only): {rf_data['min_rssi']} / {rf_data['avg_rssi']} / {rf_data['max_rssi']} dBm (Min/Avg/Max)")
        print(f"   📊 LQI (Success only): {rf_data['min_lqi']} / {rf_data['avg_lqi']} / {rf_data['max_lqi']} (Min/Avg/Max)")
    else:
        print(f"   📡 RSSI (Success only): N/A (no successful packets)")
        print(f"   📊 LQI (Success only): N/A (no successful packets)")
    
    # CCA Competition Rate
    if tx_ended > 0:
        cca_competition = (tx_cca_busy / tx_ended) * 100
        print(f"   🚦 CCA Competition: {cca_competition:.5f}% ({tx_cca_busy}/{tx_ended})")
    else:
        print(f"   🚦 CCA Competition: N/A (no TX packets)")
    
    print()


# Tính PER, RF metrics và CCA Competition cho từng kit
print("🔴 KẾT QUẢ PER, RF QUALITY VÀ CCA COMPETITION CHO TỪNG KIT:")
print("=" * 100)


kit_results = []
total_rx_corrupted = 0
total_rx_ended = 0
total_tx_cca_busy = 0
total_tx_ended = 0


all_kit_ids = set(list(kit_per_data.keys()) + list(kit_tx_cca_data.keys()))


for kit_id in all_kit_ids:
    per_data = kit_per_data.get(kit_id, {'RX_CORRUPTED': 0, 'RX_ENDED': 0})
    rf_data = kit_rf_data.get(kit_id, {})
    tx_cca_data = kit_tx_cca_data.get(kit_id, {'TX_CCA_BUSY': 0, 'TX_ENDED': 0})
    
    rx_corrupted = per_data['RX_CORRUPTED']
    rx_ended = per_data['RX_ENDED']
    total_rx = rx_corrupted + rx_ended
    
    tx_cca_busy = tx_cca_data['TX_CCA_BUSY']
    tx_ended = tx_cca_data['TX_ENDED']
    total_tx = tx_cca_busy + tx_ended
    
    # Tính PER cho kit này
    if total_rx > 0:
        per = rx_corrupted / total_rx
        per_percent = per * 100
        success_rate = (rx_ended / total_rx) * 100
    else:
        per_percent = 0
        success_rate = 0
    
    # THÊM MỚI: Tính CCA Competition cho kit này
    if tx_ended > 0:
        cca_competition = (tx_cca_busy / tx_ended) * 100
    else:
        cca_competition = 0
    
    # Lấy RF metrics từ successful packets (RX_ENDED) - CHỈ TỪ DICTIONARY RF_DATA
    if rf_data and rx_ended > 0:
        rssi_min = rf_data['min_rssi']
        rssi_max = rf_data['max_rssi']
        rssi_avg = rf_data['avg_rssi']
        lqi_min = rf_data['min_lqi']
        lqi_max = rf_data['max_lqi']
        lqi_avg = rf_data['avg_lqi']
    else:
        rssi_min = rssi_max = rssi_avg = 0
        lqi_min = lqi_max = lqi_avg = 0
    
    # Đánh giá chất lượng signal dựa trên RSSI trung bình của successful packets
    signal_quality = "Unknown"
    if rssi_avg != 0:
        if rssi_avg >= -50:
            signal_quality = "Excellent"
        elif rssi_avg >= -60:
            signal_quality = "Very Good"
        elif rssi_avg >= -70:
            signal_quality = "Good"
        elif rssi_avg >= -80:
            signal_quality = "Fair"
        else:
            signal_quality = "Poor"
    
    # THÊM MỚI: Đánh giá mức độ cạnh tranh CCA
    cca_level = "Unknown"
    if tx_ended > 0:
        if cca_competition <= 1:
            cca_level = "Excellent"
        elif cca_competition <= 5:
            cca_level = "Good"
        elif cca_competition <= 10:
            cca_level = "Fair"
        elif cca_competition <= 20:
            cca_level = "Poor"
        else:
            cca_level = "Critical"
    
    # Lưu kết quả
    kit_results.append({
        'Kit_ID': kit_id,
        'RX_CORRUPTED': rx_corrupted,
        'RX_ENDED': rx_ended,
        'Total_RX': total_rx,
        'PER_Percent': round(per_percent, 5),
        'Success_Rate': round(success_rate, 5),
        # THÊM MỚI: TX và CCA metrics
        'TX_CCA_BUSY': tx_cca_busy,
        'TX_ENDED': tx_ended,
        'Total_TX': total_tx,
        'CCA_Busy_Percent': round(cca_competition, 5),
        'CCA_Level': cca_level,
        # RSSI/LQI metrics CHỈ TỪ successful packets
        'RSSI_Min': rssi_min,
        'RSSI_Max': rssi_max,
        'RSSI_Avg': rssi_avg,
        'LQI_Min': lqi_min,
        'LQI_Max': lqi_max,
        'LQI_Avg': lqi_avg,
        'Signal_Quality': signal_quality,
        'Success_Packets_Count': rf_data.get('success_count', 0) if rf_data else 0
    })
    
    # Cộng dồn cho tổng
    total_rx_corrupted += rx_corrupted
    total_rx_ended += rx_ended
    total_tx_cca_busy += tx_cca_busy
    total_tx_ended += tx_ended
    
    print(f"📱 Kit ID: {kit_id}")
    print(f"   🔴 PER: {per_percent:.5f}%")
    print(f"   ✅ Success Rate: {success_rate:.5f}%")
    print(f"   🚦 CCA Competition: {cca_competition:.5f}% ({cca_level})")
    print(f"   📊 Signal Quality: {signal_quality}")
    print(f"   📡 RSSI (Success packets only): {rssi_min} / {rssi_avg} / {rssi_max} dBm")
    print(f"   📊 LQI (Success packets only): {lqi_min} / {lqi_avg} / {lqi_max}")
    print(f"   📦 RX Packets: {rx_corrupted} corrupted, {rx_ended} success, {total_rx} total")
    print(f"   📡 TX Packets: {tx_cca_busy} CCA busy, {tx_ended} success, {total_tx} total")
    print(f"   🎯 Success packets used for RF calculation: {rf_data.get('success_count', 0) if rf_data else 0}")
    print()


# Tính tổng cộng
total_rx_packets = total_rx_corrupted + total_rx_ended
total_tx_packets = total_tx_cca_busy + total_tx_ended


if total_rx_packets > 0:
    total_per = (total_rx_corrupted / total_rx_packets) * 100
    total_success = (total_rx_ended / total_rx_packets) * 100
else:
    total_per = 0
    total_success = 0


if total_tx_ended > 0:
    total_cca_competition = (total_tx_cca_busy / total_tx_ended) * 100
else:
    total_cca_competition = 0


print("🌟 TỔNG KẾT CHUNG:")
print("=" * 80)
print(f"📊 Tổng số kit: {len(all_kit_ids)}")
print(f"📦 Tổng RX packets: {total_rx_packets:,}")
print(f"❌ Tổng RX_CORRUPTED: {total_rx_corrupted:,}")
print(f"✅ Tổng RX_ENDED: {total_rx_ended:,}")
print(f"🔴 PER trung bình: {total_per:.5f}%")
print(f"✅ Success rate trung bình: {total_success:.5f}%")
print(f"📡 Tổng TX packets: {total_tx_packets:,}")
print(f"🚦 Tổng TX_CCA_BUSY: {total_tx_cca_busy:,}")
print(f"📡 Tổng TX_ENDED: {total_tx_ended:,}")
print(f"🚦 CCA Competition trung bình: {total_cca_competition:.5f}%")
print(f"📡 RSSI/LQI chỉ được tính từ {total_rx_ended:,} successful packets")


# Tạo DataFrame cho dễ đọc - THÊM CCA COMPETITION
df_basic = pd.DataFrame(kit_results)[['Kit_ID', 'RX_CORRUPTED', 'RX_ENDED', 'Total_RX', 'PER_Percent', 'Success_Rate']]
df_basic['PER_Percent'] = df_basic['PER_Percent'].round(5)
df_basic['Success_Rate'] = df_basic['Success_Rate'].round(5)


# THÊM MỚI: Bảng CCA Competition
df_cca = pd.DataFrame(kit_results)[['Kit_ID', 'TX_CCA_BUSY', 'TX_ENDED', 'Total_TX', 'CCA_Busy_Percent', 'CCA_Level']]
df_cca['CCA_Busy_Percent'] = df_cca['CCA_Busy_Percent'].round(5)


df_rf_success = pd.DataFrame(kit_results)[['Kit_ID', 'RSSI_Min', 'RSSI_Avg', 'RSSI_Max', 
                                          'LQI_Min', 'LQI_Avg', 'LQI_Max']]


print(f"\n📋 BẢNG 1: PER VÀ SUCCESS RATE:")
print("=" * 100)
print(df_basic.to_string(index=False))


print(f"\n🚦 BẢNG 2: CCA Busy Rate:")
print("=" * 100)
print(df_cca.to_string(index=False))


print(f"\n📡 BẢNG 3: RF METRICS:")
print("=" * 100)
print(df_rf_success.to_string(index=False))


# Thống kê kit có metrics cao nhất và thấp nhất
if len(kit_results) > 0:
    best_kit = min(kit_results, key=lambda x: x['PER_Percent'])
    worst_kit = max(kit_results, key=lambda x: x['PER_Percent'])
    
    # THÊM MỚI: Kit có CCA Competition tốt nhất và kém nhất
    cca_kits = [k for k in kit_results if k['TX_ENDED'] > 0]
    if cca_kits:
        best_cca_kit = min(cca_kits, key=lambda x: x['CCA_Busy_Percent'])
        worst_cca_kit = max(cca_kits, key=lambda x: x['CCA_Busy_Percent'])
    
    # Kit có RSSI tốt nhất và kém nhất (chỉ từ successful packets)
    rssi_kits = [k for k in kit_results if k['RSSI_Avg'] != 0]
    if rssi_kits:
        best_rssi_kit = max(rssi_kits, key=lambda x: x['RSSI_Avg'])
        worst_rssi_kit = min(rssi_kits, key=lambda x: x['RSSI_Avg'])
    
    # Kit có LQI tốt nhất và kém nhất (chỉ từ successful packets)
    lqi_kits = [k for k in kit_results if k['LQI_Avg'] != 0]
    if lqi_kits:
        best_lqi_kit = max(lqi_kits, key=lambda x: x['LQI_Avg'])
        worst_lqi_kit = min(lqi_kits, key=lambda x: x['LQI_Avg'])
    
    print(f"\n🏆 THỐNG KÊ ĐÁNG CHÚ Ý:")
    print("=" * 80)
    print(f"✅ Kit PER tốt nhất: {best_kit['Kit_ID']} (PER: {best_kit['PER_Percent']:.5f}%)")
    print(f"❌ Kit PER kém nhất: {worst_kit['Kit_ID']} (PER: {worst_kit['PER_Percent']:.5f}%)")
    
    if cca_kits:
        print(f"🚦 Kit CCA Competition tốt nhất: {best_cca_kit['Kit_ID']} (CCA: {best_cca_kit['CCA_Busy_Percent']:.5f}%)")
        print(f"🚦 Kit CCA Competition kém nhất: {worst_cca_kit['Kit_ID']} (CCA: {worst_cca_kit['CCA_Busy_Percent']:.5f}%)")
    
    if rssi_kits:
        print(f"📡 Kit RSSI tốt nhất (success): {best_rssi_kit['Kit_ID']} (RSSI: {best_rssi_kit['RSSI_Avg']} dBm)")
        print(f"📡 Kit RSSI kém nhất (success): {worst_rssi_kit['Kit_ID']} (RSSI: {worst_rssi_kit['RSSI_Avg']} dBm)")
    
    if lqi_kits:
        print(f"📊 Kit LQI tốt nhất (success): {best_lqi_kit['Kit_ID']} (LQI: {best_lqi_kit['LQI_Avg']})")
        print(f"📊 Kit LQI kém nhất (success): {worst_lqi_kit['Kit_ID']} (LQI: {worst_lqi_kit['LQI_Avg']})")


# Kiểm tra dữ liệu
print(f"\n🔍 KIỂM TRA DỮ LIỆU:")
print("=" * 60)


no_success_kits = [result['Kit_ID'] for result in kit_results if result['RX_ENDED'] == 0]
if no_success_kits:
    print(f"⚠️ Các kit không có RX_ENDED packets: {no_success_kits}")


no_tx_kits = [result['Kit_ID'] for result in kit_results if result['TX_ENDED'] == 0]
if no_tx_kits:
    print(f"⚠️ Các kit không có TX_ENDED packets: {no_tx_kits}")


high_per_kits = [result['Kit_ID'] for result in kit_results if result['PER_Percent'] > 10.0]
if high_per_kits:
    print(f"🚨 Các kit có PER > 10%: {high_per_kits}")
else:
    print("✅ Tất cả kit đều có PER ≤ 10%")


# THÊM MỚI: Kiểm tra CCA Competition cao
high_cca_kits = [result['Kit_ID'] for result in kit_results if result['CCA_Busy_Percent'] > 20.0]
if high_cca_kits:
    print(f"🚨 Các kit có CCA Competition > 20%: {high_cca_kits}")
else:
    print("✅ Tất cả kit đều có CCA Competition ≤ 20%")


poor_signal_kits = [result['Kit_ID'] for result in kit_results if result['Signal_Quality'] == 'Poor']
if poor_signal_kits:
    print(f"📡 Các kit có signal quality kém: {poor_signal_kits}")
else:
    print("✅ Tất cả kit đều có signal quality >= Fair")


# THÊM MỚI: Thống kê CCA Level
print(f"\n📊 PHÂN TÍCH MỨC ĐỘ CẠNH TRANH CCA:")
print("=" * 70)


cca_level_count = {}
for result in kit_results:
    if result['TX_ENDED'] > 0:  # Chỉ tính kit có TX data
        level = result['CCA_Level']
        if level not in cca_level_count:
            cca_level_count[level] = 0
        cca_level_count[level] += 1


total_tx_kits = len([k for k in kit_results if k['TX_ENDED'] > 0])
for level, count in cca_level_count.items():
    percentage = (count / total_tx_kits) * 100 if total_tx_kits > 0 else 0
    print(f"   {level}: {count} kit(s) ({percentage:.1f}%)")


print(f"\n💡 QUAN TRỌNG:")
print("=" * 60)
print("📡 RSSI và LQI chỉ được tính từ RX_ENDED packets (successful reception)")
print("🎯 Điều này đảm bảo đánh giá chính xác chất lượng liên kết thực tế")
print("❌ RX_CORRUPTED packets bị loại bỏ khỏi tính toán RF metrics")
print("🚦 CCA Competition = TX_CCA_BUSY/TX_ENDED đo mức độ cạnh tranh truy cập môi trường")
print("📊 CCA Competition thấp = môi trường ít cạnh tranh, truyền dữ liệu hiệu quả")


# Đóng kết nối
conn.close()


print(f"\n✅ Hoàn thành phân tích PER, RF Quality và CCA Competition cho {len(all_kit_ids)} kit!")


📊 Phân tích PER, RF Quality và CCA Competition cho từng kit từ 2025-06-14 15:55:22.736839+07:00 đến 2025-06-15 23:21:37.408346+07:00
🔍 Chỉ tính RSSI/LQI từ RX_ENDED packets (successful reception)
📡 CCA Competition = TX_CCA_BUSY/TX_ENDED (mức độ cạnh tranh truy cập môi trường)
📈 Thống kê packets, RF Quality và CCA Competition theo từng kit:
------------------------------------------------------------------------------------------
📱 Kit 842E14FFFEF64B6B:
   📦 RX_CORRUPTED: 1
   📦 RX_ENDED: 713
   📦 Total RX: 714
   📡 TX_CCA_BUSY: 42
   📡 TX_ENDED: 452
   📡 Total TX: 494
   📡 RSSI (Success only): -63 / -26.19 / -8 dBm (Min/Avg/Max)
   📊 LQI (Success only): 148 / 254.05 / 255 (Min/Avg/Max)
   🚦 CCA Competition: 9.29204% (42/452)

📱 Kit 842E14FFFEF64BFE:
   📦 RX_CORRUPTED: 6
   📦 RX_ENDED: 7,402
   📦 Total RX: 7,408
   📡 TX_CCA_BUSY: 135
   📡 TX_ENDED: 6,915
   📡 Total TX: 7,050
   📡 RSSI (Success only): -67 / -23.98 / -9 dBm (Min/Avg/Max)
   📊 LQI (Success only): 132 / 251.83 / 255 (Min/Av